# Contents
1. Introduction
2. Setup
3. Using `dataclass`
4. Using `make_dataclass` with fields
5. Checkpoint and retrieval


# 1. Introduction
Dataclasses provide decorator and functions for automatically adding generated special methods such as `__init__()` and `__repr__()`

# 2. Setup

In [1]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
from ib_insync import util
import pandas as pd
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    pd.options.display.float_format = '{:,.2f}'.format # set float precision with comma

# 3. Using `dataclass`

## Generic dataclass

In [2]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    """Class for keeping track of an item in inventory."""
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

In [3]:
first_item = InventoryItem(name='First', unit_price=2.0, quantity_on_hand=5)
first_item

InventoryItem(name='First', unit_price=2.0, quantity_on_hand=5)

## Building objects from IBKR pickles

In [13]:
# PRIMARY IMPORTS
from pathlib import Path
import sys
import logging 
from types import SimpleNamespace

import yaml
import json 


In [14]:
# SETUPS

# .choose market and connection
MARKET = 'snp'
CONNECTION = 'live'
MKT_CONN = MARKET.lower() + '_' + CONNECTION.lower()

# .set the paths
root_path = Path.cwd().parent.parent.joinpath('experiments')
log_path = root_path.joinpath('log', MKT_CONN + '.log')

# .set the notebook for imports
if str(root_path) not in sys.path:
    print(f'Root path not there in sys. Adding {str(root_path)}')
    sys.path.append(str(root_path))

# .objectify var.yml
var_file = root_path.joinpath('var.yml')
with open(var_file, "rb") as f:
    var_dict = yaml.safe_load(f)

vars = json.loads(json.dumps(var_dict), object_hook=lambda d: SimpleNamespace(**d))

# .set logs
util.logToFile(path=log_path, level=logging.WARNING)
with open(log_path, 'w'):
    pass

In [20]:
# Load the base files

data_path = root_path.joinpath('data', MARKET.lower())

""" 
# .code automates making df from all pickles.
#   however pylance doesn't recognize the saved variables

from os import listdir
import numpy as np 
fs = listdir(data_path)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(data_path.joinpath(f))")
np.sort(np.array(files)) """

df_symlots = pd.read_pickle(data_path.joinpath('df_symlots.pkl'))
df_ohlcs = pd.read_pickle(data_path.joinpath('df_ohlcs.pkl'))
df_chains = pd.read_pickle(data_path.joinpath('df_chains.pkl'))
df_unds = pd.read_pickle(data_path.joinpath('df_unds.pkl'))

In [21]:
# ** IMPORTS
from dataclasses import dataclass, make_dataclass, field
import pandas as pd
import numpy as np
from datetime import datetime, timezone

### using @dataclass

In [22]:
# Make objects using dataclass
@dataclass
class Scrips:
    symlots: pd.DataFrame = df_symlots
    ohlcs: pd.DataFrame = df_ohlcs
    chains: pd.DataFrame = df_chains
    unds: pd.DataFrame = df_unds

In [27]:
scrips = Scrips()
scrips.symlots.head()

,symbol,secType,expiryM,lot,exchange,currency,contract
0,MMM,STK,None,100,SMART,USD,"Contract(secType='STK', conId=9720, symbol='MM..."
1,ABT,STK,None,100,SMART,USD,"Contract(secType='STK', conId=4065, symbol='AB..."
2,ABBV,STK,None,100,SMART,USD,"Contract(secType='STK', conId=118089500, symbo..."
3,ACN,STK,None,100,SMART,USD,"Contract(secType='STK', conId=67889930, symbol..."
4,ATVI,STK,None,100,SMART,USD,"Contract(secType='STK', conId=52424577, symbol..."


# 4. Using `make_dataclass` with fields

In [39]:
fields = [('float_val', float, field(default=np.nan)),
          ('df', pd.DataFrame, field(default_factory=pd.DataFrame)),
          ('int_val', int, field(default_factory=int)),
          ('time_val', datetime, field(default=datetime.now(timezone.utc)))]

Aclass = make_dataclass('Aclass', fields)



In [40]:
an_instance=Aclass()
an_instance.time_val

datetime.datetime(2021, 12, 20, 0, 50, 28, 20475, tzinfo=datetime.timezone.utc)

# 5. `dill` checkpoints

In [50]:
temp_path = root_path.parent.joinpath('learn', 'temp')
temp_file = temp_path.joinpath('dill_temp.dump')

In [51]:
import dill
dill.dump(scrips, open(temp_file, 'wb'))

In [54]:
# Retrieve the data
loaded_scrip = dill.load(open(temp_file, 'rb'))
loaded_scrip.symlots.head()

,symbol,secType,expiryM,lot,exchange,currency,contract
0,MMM,STK,None,100,SMART,USD,"Contract(secType='STK', conId=9720, symbol='MM..."
1,ABT,STK,None,100,SMART,USD,"Contract(secType='STK', conId=4065, symbol='AB..."
2,ABBV,STK,None,100,SMART,USD,"Contract(secType='STK', conId=118089500, symbo..."
3,ACN,STK,None,100,SMART,USD,"Contract(secType='STK', conId=67889930, symbol..."
4,ATVI,STK,None,100,SMART,USD,"Contract(secType='STK', conId=52424577, symbol..."
